# Import Preprocess Data, Libraries

In [125]:
import pandas as pd
from pyArango.connection import Connection

In [152]:
df = pd.read_csv('./processed_data.csv', low_memory=False)
df.head()

,timestamp,userId,hw_released,hw_device_id,hw_cpu_name,hw_screen_aspect_ratio,hw_screen_pixel_density,system_packages,network_wifi_mac_address,hw_screen_resolution,...,hw_ram_total,hw_cpu_min_speed,hw_storage_total,list_of_wifi,hw_camera_front_max_photo_resolution,hw_camera_back_max_video_resolution,ringtone,hw_camera_number_camera,hw_camera_back_max_photo_resolution,hw_camera_front_max_video_resolution
0,1596244856,170425000002238,Tue Jul 07 08:50:39 GMT+07:00 2020,f21ebb2f9064b16e,exynos9810,2,420 Dpi,['com.samsung.android.provider.filterprovider'...,12:2F:14:33:9D:E7,1080x2400,...,"7,94 GB",455000Hz,118 GB,[''],NaN,NaN,NaN,NaN,NaN,NaN
1,1596244830,191113586003363,Thu Jul 02 12:19:14 GMT+07:00 2020,13cf342d200936c8,hi6250,1,320 Dpi,"['com.huawei.hifolder', 'com.android.cts.priv....",A8:E5:44:0E:BD:BB,1200x1920,...,"2,88 GB",480000Hz,"25,23 GB","['Dinh1', 'MaiCa', 'NhuQuynh', 'Support', 'Nam...",NaN,NaN,NaN,NaN,NaN,NaN
2,1596244888,200413000001404,Thu Dec 07 11:24:05 GMT+07:00 2017,b41f717314855504,sc8830,1,320 Dpi,"['com.monotype.android.font.rosemary', 'com.se...",AC:AF:B9:61:36:C8,720x1280,...,"1,33 GB",768000Hz,"4,79 GB","['Xiaomi_70BE_8F41', 'COM TAM SONG ANH', 'Redm...",2576x1932,3264x2448,"['Basic Bell', 'Basic Tone', 'Beep Once', 'Bee...",2.0,3264x2448,2576x1932
3,1596244831,200428000037320,Wed Apr 22 07:53:33 GMT+07:00 2020,8d36707898c6955d,samsungexynos9810,2,420 Dpi,['com.samsung.android.provider.filterprovider'...,24:18:1D:FE:E3:E0,1080x2220,...,"5,89 GB",455000Hz,120 GB,[''],NaN,NaN,NaN,NaN,NaN,NaN
4,1596244849,190309000002524,Thu Mar 12 12:05:05 GMT+07:00 2020,af1934dafd8aba01,mt6757,1,420 Dpi,['com.samsung.android.provider.filterprovider'...,A4:6C:F1:C3:34:4E,1080x1920,...,3.92 GB,247000Hz,26.75 GB,"['NamCoffee&Milktea', 'YÊU MỘT NGƯỜI', 'Studio...",4608x3456,4128x3096,NaN,2.0,4128x3096,4608x3456


# Connect to ArangoDB, DB and collections

In [137]:
conn = Connection(username='root', password='ngph')
db = conn['final']
users = db['users']
devices = db['devices']
user_device = db['user_device']
device_deviceModel = db['device_devicemodel']
device_model = db['device_model']

# Insert data into ArangoDB

## Add users

In [165]:
unique_users = df['userId'].unique()

In [19]:
for uu in unique_users:
    users.createDocument({
        '_key': uu
    }).save()

## Add latest devices info

In [141]:
devices_latest = df[df.groupby(['hw_device_id'])['timestamp'].transform(max) == df['timestamp']]

In [146]:
cols_exclude_devices = set(df.columns) - set(['hw_device_id', 'installed_packages', 'system_packages', 'list_of_wifi', 'ringtone'])
for _, row in devices_latest.iterrows():
    nd = devices.createDocument({
        '_key': row['hw_device_id'],
        'installed_packages': [x.strip() for x in row['installed_packages'].split(',')] if type(row['installed_packages']) is not float else 'nan',
        'system_packages': [x.strip() for x in row['system_packages'].split(',')] if type(row['system_packages']) is not float else 'nan',
        'list_of_wifi': [k.strip().replace('"', '') for k in row['list_of_wifi'].split(',')] if type(row['list_of_wifi']) is not float else 'nan',
        'ringtone': [k.strip() for k in row['ringtone'].split(',')] if type(row['ringtone']) is not float else 'nan'
    })
    for c in cols_exclude_devices:
        nd[c] = str(row[c])
    nd.save()

===
Unable to establish connection, perhaps arango is not running.
===


KeyboardInterrupt: 

## Add users - devices

In [97]:
cols_exclude_users_devices = set(df.columns) - set(['userId', 'hw_device_id', 'installed_packages', 'system_packages'])
for _, row in df.iterrows():
    ud = user_device.createDocument({
        '_from': 'users/' + str(row['userId']),
        '_to': 'devices/' + str(row['hw_device_id'])
    })
    for c in cols_exclude_users_devices:
        ud[c] = str(row[c])
    ud.save()

## Devices model

In [115]:
unique_device_model = df['hw_device_model'].unique()
inserted = []
for uq in unique_device_model:
    formatted = '_'.join(x for x in uq.split(' '))
    if formatted not in inserted:
        inserted.append(formatted)
        device_model.createDocument({
            '_key': formatted
        }).save()

## Device - DeviceModel

In [117]:
for _, row in df.iterrows():
    formatted_device_model = '_'.join(x for x in row['hw_device_model'].split(' '))
    new_ddm = device_deviceModel.createDocument({
        '_from': 'devices/' + str(row['hw_device_id']),
        '_to': 'device_model/' + formatted_device_model,
        'timestamp': str(row['timestamp']),
        'userId': str(row['userId'])
    })
    new_ddm.save()